In [1]:
import dev.biserman.planet.language.Language
import dev.biserman.planet.language.Segment
import dev.biserman.planet.language.SyllableConstructor
import dev.biserman.planet.language.InventoryTransformation
import dev.biserman.planet.language.Manner
import dev.biserman.planet.language.SegmentType


In [2]:
import dev.biserman.planet.language.Glide
import dev.biserman.planet.language.Place
import dev.biserman.planet.language.SegmentData
import dev.biserman.planet.utils.toWeightedBag
import kotlin.random.Random

SyllableConstructor.languageFile = """E:\Users\Winggar\source\repos\Planet\planet\english.json"""
SyllableConstructor.phonemeFile = """E:\Users\Winggar\source\repos\Planet\planet\phonemes.json"""

val basicPhonemes = "ptksmnljw"
val basicTransformation: InventoryTransformation = { inventory ->
    inventory.plus(basicPhonemes.mapNotNull { SyllableConstructor.segments[it.toString()] })
}

fun (Set<Segment>).addSet(
    from: (SegmentData) -> Boolean,
    to: (SegmentData) -> SegmentData,
    condition: Boolean = true
): Set<Segment> {
    if (!condition) return this

    val new = this
        .filter { from(it.data) }
        .map { to(it.data) }
    val matching = SyllableConstructor.segments.values.filter { it.data in new }
    return this.plus(matching)
}

val random = Random(System.currentTimeMillis())

val placeWeights = mapOf(
    Place.LABIAL to 40,
    Place.DENTAL to 5,
    Place.ALVEOLAR to 50,
    Place.POSTALVEOLAR to 35,
    Place.PALATAL to 25,
    Place.VELAR to 45,
    Place.UVULAR to 5,
    Place.GLOTTAL to 50,
)

val inversePlaceWeights = (placeWeights.values.max()).let { maxWeight ->
    placeWeights.mapValues { (_, weight) -> maxWeight - weight + 1 }
}

//val affricates = mapOf(
//    'ɸ' to Pair('p', 10),
//    'β' to Pair('b', 10),
//    'f' to Pair('p', 50),
//    'v' to Pair('b', 50),
//    'θ' to Pair('t', 10),
//    'ð' to Pair('d', 10),
//    's' to Pair('t', 100),
//    'z' to Pair('d', 100),
//    'ʃ' to Pair('t', 250),
//    'ʒ' to Pair('d', 250),
//    'x' to Pair('k', 100)
//)

fun center(x: Double) = 4 * (x - 0.5).pow(3) + 0.5

val inventoryTransformations = listOf<Pair<InventoryTransformation, Int>>(
    { inventory: Set<Segment> -> // add voicing
        val chance = random.nextDouble()
        val newInventory = inventory.let {
            if (chance <= 0.9) {
                inventory.addSet(
                    from = { it.manner == Manner.PLOSIVE },
                    to = { it.copy(voiced = true) },
                    condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
            } else it
        }.let {
            if (chance <= 0.4 || chance > 0.9) {
                inventory.addSet(
                    from = { it.manner == Manner.FRICATIVE },
                    to = { it.copy(voiced = true) },
                    condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
            } else it
        }
        newInventory
    } to 600,
    { inventory: Set<Segment> -> // add aspiration
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(isAspirated = true, voiced = null) },
            condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
    } to 400,
    { inventory: Set<Segment> -> // add ejectives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(isEjective = true, voiced = null) })
    } to 100,

    { inventory: Set<Segment> -> // add plosives from fricatives
        inventory.addSet(
            from = { it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.PLOSIVE) })
    } to 300,
    { inventory: Set<Segment> -> // add fricatives from plosives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE },
            to = { it.copy(manner = Manner.FRICATIVE) })
    } to 300,

    { inventory: Set<Segment> -> // add rhotic
        inventory.plus(SyllableConstructor.segments["ɹ"]!!)
    } to 500,
    { inventory: Set<Segment> -> // add implosives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.IMPLOSIVE) })
    } to 100,
    { inventory: Set<Segment> -> // add clicks
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.CLICK, voiced = null) })
    } to 100,
    { inventory: Set<Segment> -> // add nasals
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.NASAL, voiced = null) })
    } to 250,
    { inventory: Set<Segment> -> // remove randomly
        inventory.filter { random.nextDouble() <= center(it.prevalence).pow(0.33) }.toSet()
    } to 500,

    affricate@{ inventory: Set<Segment> -> // add affricates
        val fricative =
            inventory.filter { it.data.manner == Manner.FRICATIVE && it.data.place != Place.GLOTTAL }
                .randomOrNull(random) ?: return@affricate inventory
        val glide = Glide.from(fricative.data, false)
        val affricate = if (random.nextDouble() <= 0.75) {
            val place = when (fricative.data.place) {
                Place.LABIODENTAL -> Place.BILABIAL
                Place.POSTALVEOLAR -> Place.ALVEOLAR
                Place.PALATAL -> Place.ALVEOLAR
                Place.DENTAL -> Place.ALVEOLAR
                else -> fricative.data.place
            }
            fricative.copyData { it.copy(place = place, manner = Manner.PLOSIVE, consonantGlide = glide) }
        } else {
            val plosive =
                inventory
                    .filter {
                        it.data.manner == Manner.PLOSIVE &&
                                it.data.place != Place.GLOTTAL &&
                                it.data.isAspirated == false &&
                                it.data.isEjective == false &&
                                it.data.voiced == fricative.data.voiced
                    }
                    .randomOrNull(random) ?: return@affricate inventory
            plosive.copyData { it.copy(consonantGlide = glide) }
        }
        inventory.plus(affricate)
    } to 300,

    glideColoredSet@{ inventory: Set<Segment> -> // add a glide-colored set
        val chance = random.nextDouble()
        val glide = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.SEMIVOWEL || it.data.manner == Manner.LIQUID }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSet inventory

        val newInventory = inventory.let {
            if (chance <= 0.67) {
                it.plus(it.filter { it.data.manner == Manner.FRICATIVE }
                    .map { it.copyData { it.copy(offGlide = Glide.from(glide.data, false)) } })
            } else it
        }.let {
            if (chance <= 0.33 || chance > 0.67) {
                it.plus(it.filter { it.data.manner == Manner.PLOSIVE }
                    .filter { it.data.place != Place.GLOTTAL }
                    .map { it.copyData { it.copy(offGlide = Glide.from(glide.data, false)) } })
            } else it
        }

        newInventory
    } to 200,

    glideColoredSingle@{ inventory: Set<Segment> ->
        val glide = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.SEMIVOWEL || it.data.manner == Manner.LIQUID }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSingle inventory

        val plosive = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.PLOSIVE }
            .filter { it.data.place != Place.GLOTTAL }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSingle inventory

        inventory.plus(plosive.copyData { it.copy(offGlide = Glide.from(glide.data, false)) })
    } to 300,

    { inventory: Set<Segment> -> // tʃ, dʒ
        if (inventory.any { it.data.place == Place.ALVEOLAR && it.data.manner == Manner.PLOSIVE }) {
            val tʃ = SyllableConstructor.segments["t"]!!.copyData {
                it.copy(
                    consonantGlide = Glide(
                        Place.POSTALVEOLAR,
                        Manner.FRICATIVE,
                        false
                    )
                )
            }
            val dʒ = SyllableConstructor.segments["d"]!!.copyData {
                it.copy(
                    consonantGlide = Glide(
                        Place.POSTALVEOLAR,
                        Manner.FRICATIVE,
                        false
                    )
                )
            }

            if (inventory.any { it.data.place == Place.ALVEOLAR && it.data.manner == Manner.PLOSIVE && it.data.voiced == true }) {
                inventory.plus(listOf(tʃ, dʒ))
            } else {
                inventory.plus(tʃ)
            }
        } else {
            inventory
        }
    } to 200,
).plus(placeWeights.map { (place, weight) ->
    { inventory: Set<Segment> ->
        inventory.addSet(
            from = { it.type == SegmentType.CONSONANT },
            to = { it.copy(place = place) })

    } to weight * 3
}).plus(inversePlaceWeights.map { (place, weight) ->
    { inventory: Set<Segment> ->
        inventory.filter { it.data.place != place }.toSet()
    } to weight
})

val bag = inventoryTransformations.toWeightedBag(random) { it.second }

fun generateConsonants() = basicTransformation(setOf()).let {
    (1..15).fold(it) { acc, _ -> bag.grab()!!.first.invoke(acc) }
}.sortedBy { SyllableConstructor.segments.keys.indexOf(it.symbol) }

for (_1 in 0..10) {
    val testLanguage = generateConsonants()
    println("${testLanguage.size} phonemes: ${testLanguage.map { it.display }}")
}


15 phonemes: [m, n, ŋ, p, t, tʷ, k, k͡s, ɡ, ɡˡ, f, s, w, ɹ, l]
24 phonemes: [m, n, ŋ, p, p͡s, p?, t͡ʃ, t͡s, t?, k, k?, b, b?, ɡ, ɡ?, f, f?, s, s?, v, z, z?, j, l]
25 phonemes: [m, n, ŋ, p, p?, t, t͡ʃ, t?, k, k?, b, b?, d, dʷ, d͡ʒ, d?, ɡ, ɡ?, s, s?, z, z?, j, w, l]
21 phonemes: [m, n, ŋ, p, pʷ, t, tʷ, t͡ʃ, k, kʷ, d, d͡ʒ, ɡ, s, sʷ, z, zʷ, j, w, ɹ, l]
13 phonemes: [m, n, p, t, t͡ʃ, k, kʷ, f, s, j, w, ɹ, l]
15 phonemes: [m, n, ŋ, p, t, tˡ, t͡s, k, k?, f, s, v, j, ɹ, l]
26 phonemes: [m, n, ŋ, p, p?, t, tʷ, t?, k, kʷ, k?, b, b?, d?, ɡ, ɡ?, f, fʷ, s, v, vʷ, z, zʷ, j, w, l]
16 phonemes: [m, n, ŋ, p, pˡ, t, t͡ʃ, k, s, ʃ, z, ʒ, j, w, ɹ, l]
19 phonemes: [m, n, ŋ, p, pʷ, pʳ, t, tˡ, t͡s, tʷ, tʳ, k, kʳ, f, s, j, w, ɹ, l]
19 phonemes: [m, n, ŋ, p, t, t͡ʃ, k, b, d, d͡ʒ, ɡ, f, s, v, z, ʔ, w, ɹ, l]
17 phonemes: [m, n, p, t, t͡s, t?, k, k?, b, d, f, s, v, z, j, w, ɹ]


In [3]:
SyllableConstructor.segments.entries.map {
    println("${it.key} - ${it.value}")
}

m - Segment(symbol=m, data=SegmentData(type=CONSONANT, place=LABIAL, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null, nasalized=null, lengthened=null), prevalence=0.97)
n - Segment(symbol=n, data=SegmentData(type=CONSONANT, place=ALVEOLAR, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null, nasalized=null, lengthened=null), prevalence=0.97)
ŋ - Segment(symbol=ŋ, data=SegmentData(type=CONSONANT, place=VELAR, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null, nasalized=null, lengthened=null), prevalence=0.69)
p - Segment(symbol=p, data=SegmentData(type=CONSONANT, place=LABIAL, manner=PLOSIVE, voiced=false, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, conso

[kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit]

In [8]:
import dev.biserman.planet.geometry.scaleAndCoerceIn
import dev.biserman.planet.language.Depth
import dev.biserman.planet.language.Height

val vowelDistanceFeatures = listOf(
    SegmentData::depth,
    SegmentData::height,
    SegmentData::rounded,
    SegmentData::lengthened,
    SegmentData::nasalized,
    SegmentData::onGlide,
    SegmentData::offGlide,
)

inline fun <reified T : Enum<T>> center() = (enumValues<T>().size / 2.0).roundToInt()

@JvmName("vowelNonNull")
fun (Glide).vowel(isOnGlide: Boolean): Segment = (this as Glide?).vowel(isOnGlide)!!
fun (Glide?).vowel(isOnGlide: Boolean): Segment? {
    return when {
        this == null -> null
        this.manner != Manner.SEMIVOWEL -> null
        this.place == Place.PALATAL ->
            SyllableConstructor.segments["i"]!!
        this.place == Place.LABIAL ->
            if (isOnGlide) SyllableConstructor.segments["u"]
            else SyllableConstructor.segments["o"]
        else -> null
    }
}

fun (Segment?).vowelDistanceTo(other: Segment?): Int = if (this == null || other == null) 0 else
    (vowelDistanceFeatures.count { feature -> feature.get(this.data) != feature.get(other.data) }
            + (this.data.height!!.ordinal - other.data.height!!.ordinal).absoluteValue
            + (this.data.depth!!.ordinal - other.data.depth!!.ordinal).absoluteValue
            + (this.data.onGlide.vowel(true).vowelDistanceTo(other.data.onGlide.vowel(true)))
            + (this.data.offGlide.vowel(false).vowelDistanceTo(other.data.offGlide.vowel(false))))

val (SegmentData).isFrontward get() = this.depth!!.ordinal < Depth.CENTRAL.ordinal
val (SegmentData).isBackward get() = this.depth!!.ordinal > Depth.CENTRAL.ordinal
val (SegmentData).isHigh get() = this.height!!.ordinal < Height.MID.ordinal
val (SegmentData).isLow get() = this.height!!.ordinal > Height.MID.ordinal

fun generateVowels(): List<Segment> {
    val allVowels = SyllableConstructor.segments.values
        .filter { it.data.type == SegmentType.VOWEL }

    val baseVowels = allVowels
        .filter { random.nextDouble() <= it.prevalence }

    // adjusts for typo.uni-konstanz.de → Universal 1284
    fun heightNasalityAdjustment(height: Height) = (-height.ordinal.toDouble()).scaleAndCoerceIn(-6.0..0.0, 0.7..1.3)

    val nasalVowels = if (random.nextDouble() <= 0.2) {
        baseVowels.filter {
            val chance = random.nextDouble() * heightNasalityAdjustment(it.data.height!!)
            chance <= it.prevalence * 1.5
        }.map { it.copy(data = it.data.copy(nasalized = true)) }
    } else allVowels.filter {
        random.nextDouble() <= it.prevalence * 0.05 * heightNasalityAdjustment(it.data.height!!)
    }.map { it.copy(data = it.data.copy(nasalized = true)) }

    val longVowels = if (random.nextDouble() <= 0.35) {
        baseVowels.plus(nasalVowels).filter {
            val chance = random.nextDouble()
            chance <= it.prevalence * 2.0
        }.map { it.copy(data = it.data.copy(lengthened = true)) }
    } else listOf()

    val wOnGlides = if (random.nextDouble() <= 0.02) {
        baseVowels.plus(nasalVowels)
            .filter {
                val chance = random.nextDouble()
                chance <= it.prevalence * 2.0
            }
            .filter { it.symbol != "u" }
            .map { it.copy(data = it.data.copy(onGlide = Glide(Place.LABIAL, Manner.SEMIVOWEL, true))) }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "u" }
        .map { it.copy(data = it.data.copy(onGlide = Glide(Place.LABIAL, Manner.SEMIVOWEL, true))) }

    val jOnGlides = if (random.nextDouble() <= 0.02) {
        baseVowels.plus(nasalVowels)
            .filter {
                val chance = random.nextDouble()
                chance <= it.prevalence * 2.0
            }
            .filter { it.symbol != "i" }
            .map { it.copy(data = it.data.copy(onGlide = Glide(Place.PALATAL, Manner.SEMIVOWEL, true))) }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "i" }
        .map { it.copy(data = it.data.copy(onGlide = Glide(Place.PALATAL, Manner.SEMIVOWEL, true))) }

    val wOffGlides = if (random.nextDouble() <= 0.02) {
        baseVowels.plus(nasalVowels)
            .plus(jOnGlides)
            .filter {
                val chance = random.nextDouble()
                chance <= it.prevalence * 2.0
            }
            .filter { it.symbol != "o" }
            .map { it.copy(data = it.data.copy(offGlide = Glide(Place.LABIAL, Manner.SEMIVOWEL, false))) }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "o" }
        .map { it.copy(data = it.data.copy(offGlide = Glide(Place.LABIAL, Manner.SEMIVOWEL, false))) }

    val jOffGlides = if (random.nextDouble() <= 0.02) {
        baseVowels.plus(nasalVowels)
            .plus(wOnGlides)
            .filter {
                val chance = random.nextDouble()
                chance <= it.prevalence * 2.0
            }
            .filter { it.symbol != "i" }
            .map { it.copy(data = it.data.copy(offGlide = Glide(Place.PALATAL, Manner.SEMIVOWEL, false))) }
    } else allVowels.filter { random.nextDouble() <= it.prevalence * 0.01 }
        .filter { it.symbol != "i" }
        .map { it.copy(data = it.data.copy(offGlide = Glide(Place.PALATAL, Manner.SEMIVOWEL, false))) }

    return baseVowels + nasalVowels + longVowels + wOnGlides + jOnGlides + wOffGlides + jOffGlides
}

for (_1 in 0..10) {
    println(generateVowels().map { it.display })
}


[i, u, e, o, a, ao̯]
[i, u, e, o, a]
[i, u, ɪ, o, ɛ, a]
[i, u, e, ɛ, a, ĩ, ũ, ẽ, ɛ̃, ã]
[i, u, ɪ, ɛ, æ, a, ĩ, ũ, ã, iː, uː, ɪː, ɛː, aː, ĩː, ũː, ãː]
[i, u, e, o, ɛ, a]
[i, u, e, o, a, iː, uː, eː, oː, aː]
[i, u, e, a, io̯, uo̯, eo̯, ao̯]
[i, o, ɛ, a]
[i, u, e, o, a, ĩ, ũ, ẽ, õ, ã]
[i, u, e, a, ĩ, ũ, ẽ, ã, u̯e]


In [26]:
val allGlides = SyllableConstructor.segments.values
    .filter { it.data.place != Place.GLOTTAL }
    .filter {
        it.data.manner in listOf(
            Manner.FRICATIVE,
            Manner.SEMIVOWEL,
            Manner.LIQUID
        )
    }.flatMap { listOf(Glide.from(it.data, false), Glide.from(it.data, true)) }
    .toSet()
    .plus(null)

data class ConsonantSlot(
    val manner: Set<Manner> = Manner.values().toSet(),
    val place: Set<Place> = Place.values().toSet(),
    val voiced: Set<Boolean?> = setOf(true, false, null),
    val consonantGlide: Set<Glide?> = allGlides.filter { it?.manner == Manner.FRICATIVE || it?.manner == null }.toSet(),
    val offGlide: Set<Glide?> = allGlides.filter { it?.manner != Manner.FRICATIVE }.toSet()
) {
    fun getMatching(segments: List<Segment>) = segments.filter {
        it.data.type == SegmentType.CONSONANT &&
                it.data.manner in manner &&
                it.data.place in place &&
                it.data.voiced in voiced &&
                it.data.consonantGlide in consonantGlide &&
                it.data.offGlide in offGlide
    }

    companion object
}

In [27]:
import kotlin.random.nextInt

fun (ConsonantSlot.Companion).gen(consonants: List<Segment>): ConsonantSlot {
    val mainCandidates = (1..random.nextInt(1..2)).map { consonants.random() }
    val secondaryCandidates = (1..random.nextInt(1..3)).map { consonants.random() }

    return ConsonantSlot(
        manner = mainCandidates.map { it.data.manner!! }.toSet(),
        place = (mainCandidates + secondaryCandidates).map { it.data.place!! }.toSet(),
        voiced = (mainCandidates + secondaryCandidates).map { it.data.voiced }.toSet(),
        consonantGlide = mainCandidates.map { it.data.consonantGlide }.plus(null).toSet(),
        offGlide = mainCandidates.map { it.data.offGlide }.plus(null).toSet()
    )
}

In [11]:
// adapted from: https://gist.github.com/erikhuizinga/d2ca2b501864df219fd7f25e4dd000a4

import kotlin.reflect.KFunction

/**
 * Create the cartesian product of any number of sets of any size. Useful for parameterized tests
 * to generate a large parameter space with little code. Note that any type information is lost, as
 * the returned set contains list of any combination of types in the input set.
 *
 * @param sets The sets.
 */
fun <T> cartesianProduct(vararg sets: Set<T>) =
    sets
        .fold(listOf(listOf<T>())) { acc, set ->
            acc.flatMap { list -> set.map { element -> list + element } }
        }
        .toSet()


In [37]:
fun isSonoritySequenced(cluster: List<ConsonantSlot>, isOnset: Boolean): Boolean = when {
    isOnset -> cluster
        .windowed(size = 2) { (a, b) ->
            a.manner.minOf { it.ordinal } >= b.manner.maxOf { it.ordinal } &&
                    (a.manner.maxOf { it.ordinal } < Manner.PLOSIVE.ordinal || b.manner.maxOf { it.ordinal } < Manner.PLOSIVE.ordinal)
        }
        .all { it }
    else -> isSonoritySequenced(cluster.reversed(), !isOnset)
}

typealias Cluster = List<ConsonantSlot>

fun generateClusters(
    onsetMaxConsonants: Int,
    codaMaxConsonants: Int,
    fallOff: Double,
    sonoritySequencingStrictness: Double,
    consonants: List<Segment>
): Pair<List<Cluster>, List<Cluster>> {
    val onsetClusters = mutableListOf<List<ConsonantSlot>>(
        listOf(
            ConsonantSlot(
                manner = consonants.mapNotNull { it.data.manner }.toSet(),
                place = consonants.mapNotNull { it.data.place }.toSet(),
                voiced = consonants.map { it.data.voiced }.toSet(),
                consonantGlide = consonants.map { it.data.consonantGlide }.toSet(),
                offGlide = consonants.map { it.data.offGlide }.toSet()
            )
        )
    )
    val codaClusters = mutableListOf<List<ConsonantSlot>>(listOf())

    if (random.nextDouble() < 0.98) onsetClusters.add(listOf())

    for (i in 2..onsetMaxConsonants) {
        val attempts = max(1.0, onsetMaxConsonants - i * fallOff + random.nextDouble(-2.0, 1.0)).roundToInt()
        for (_j in 1..attempts) {
            while (true) {
                val newCluster =
                    if (i >= 3 && random.nextDouble() < 0.5)
                        listOf(ConsonantSlot.gen(consonants)) + onsetClusters.filter { it.size == i - 1 }.random()
                    else (1..i).map { ConsonantSlot.gen(consonants) }
                if (random.nextDouble() < sonoritySequencingStrictness.pow(1.0 / i) &&
                    !isSonoritySequenced(newCluster, true)
                ) continue
                if (newCluster.any { it.getMatching(consonants).isEmpty() }) continue

                onsetClusters.add(newCluster)
                break
            }
        }
    }

    for (i in 1..codaMaxConsonants) {
        val attempts = max(1.0, codaMaxConsonants - i * fallOff + random.nextDouble(-2.0, 1.0)).roundToInt()
        for (_j in 1..attempts) {
            val mustBeSonoritySequenced = random.nextDouble() < sonoritySequencingStrictness
            while (true) {
                val newCluster = if (i >= 3 && random.nextDouble() < 0.5)
                    codaClusters.filter { it.size == i - 1 }.random().plus(ConsonantSlot.gen(consonants))
                else (1..i).map { ConsonantSlot.gen(consonants) }
                if (mustBeSonoritySequenced && !isSonoritySequenced(newCluster, false)) {
                    continue
                }
                if (newCluster.any { it.getMatching(consonants).isEmpty() }) continue

                codaClusters.add(newCluster)
                break
            }
        }
    }

    return onsetClusters to codaClusters
}

data class Syllable(val onset: List<Segment>, val nucleus: List<Segment>, val coda: List<Segment>, val tone: Segment? = null) {
    val allConsonants by lazy { onset + coda }
    val length get() = onset.size + nucleus.size + coda.size
    override fun toString() =
        "${onset.joinToString("") { it.display }}${nucleus.joinToString("") { it.display }}${coda.joinToString("") { it.display }}"
}


In [38]:
import dev.biserman.planet.language.SyllableConstructor
import kotlin.reflect.KProperty1

class SyllableRule(val name: String, val tag: String = "", val check: (Syllable) -> Boolean)

class Prop<T, U>(val name: String, val get: (T) -> U) {
    constructor(prop: KProperty1<T, U>) : this(prop.name, prop)
}

fun generateSyllableRules(
    consonants: List<Segment>,
    vowels: List<Segment>,
    onsetClusters: List<List<ConsonantSlot>>,
    codaClusters: List<List<ConsonantSlot>>,
    maxOnset: Int,
    maxCoda: Int
): List<SyllableRule> {
    val rules = mutableListOf<SyllableRule>()

    class SidedRule(val segment: Segment, val allowedInOnset: Boolean, val allowedInCoda: Boolean)

    val possibleOnsetClusters = onsetClusters.map { cluster -> cluster.flatMap { it.getMatching(consonants) } }
    val possibleCodaClusters = codaClusters.map { cluster -> cluster.flatMap { it.getMatching(consonants) } }
//    val allPossibleOnsets = possibleOnsetClusters.flatten().toSet()
    val allPossibleCodas = possibleCodaClusters.flatten().toSet()

    if (maxOnset > 0 && maxCoda > 0) {
//        val isRareRestrictive = random.nextDouble() < 0.33
        val sidedPhonemeRules = consonants
            .filter { consonant -> (possibleOnsetClusters + possibleOnsetClusters).none { it.size == 1 && it.first() == consonant } }
            .map { consonant ->
                when (consonant.display[0]) {
                    in "ŋɳ" -> random.nextDouble().let {
                        when {
                            it >= 0.7 && consonant in allPossibleCodas -> SidedRule(consonant, false, true)
//                            it >= 0.65 && consonant in allPossibleOnsets -> SidedRule(consonant, true, false)
                            else -> SidedRule(consonant, true, true)
                        }
                    }
                    in "ptkmnlr" -> random.nextDouble().let {
                        when {
                            it >= 0.97 && consonant in allPossibleCodas -> SidedRule(consonant, false, true)
//                            it >= 0.85 && consonant in allPossibleOnsets -> SidedRule(consonant, true, false)
                            else -> SidedRule(consonant, true, true)
                        }
                    }
                    else -> random.nextDouble().let {
                        when {
                            it >= 0.95 && consonant in allPossibleCodas -> SidedRule(consonant, false, true)
//                            it >= 0.85 || isRareRestrictive && consonant in allPossibleOnsets -> SidedRule(
//                                consonant,
//                                true,
//                                false
//                            )
                            else -> SidedRule(consonant, true, true)
                        }
                    }
                }
            }.filter { !it.allowedInOnset || !it.allowedInCoda }.map { rule ->
                SyllableRule("${rule.segment.display} Sidedness (${rule.allowedInOnset}-${rule.allowedInCoda})") {
                    (rule.segment !in it.onset || rule.allowedInOnset) &&
                            (rule.segment !in it.coda || rule.allowedInCoda)
                }
            }
        rules.addAll(sidedPhonemeRules)
    }

    if (random.nextDouble() <= 0.99 && maxOnset > 1 || maxCoda > 1) {
        rules.add(SyllableRule("Offglide-Glide Adjacency") { syllable ->
            listOf(syllable.onset, syllable.coda).all { cluster ->
                cluster.windowed(size = 2) { (a, b) ->
                    a.data.offGlide == null
                            || a.data.offGlide!!.manner != Manner.SEMIVOWEL
                            || a.data.offGlide!!.place != b.data.place
                }.all { it }
            }
        })
    }

    if (random.nextDouble() <= 0.8 && consonants.any { it.data.manner == Manner.SEMIVOWEL }) {
        rules.add(SyllableRule("Semivowel-Vowel Adjacency") { syllable ->
            val onset = syllable.onset.size == 0 ||
                    (syllable.onset.last().symbol != "w" || syllable.nucleus.first().symbol != "u") &&
                    (syllable.onset.last().symbol != "j" || syllable.nucleus.first().symbol != "i")

            val coda = syllable.coda.size == 0 ||
                    (syllable.coda.first().symbol != "w" || syllable.nucleus.last().symbol != "u") &&
                    (syllable.coda.first().symbol != "j" || syllable.nucleus.last().symbol != "i")
            onset && coda
        })
    }

    while (random.nextDouble() <= 0.33) {
        val place = listOf(Syllable::onset, Syllable::coda, Syllable::allConsonants).random()
        val consonantFeature = listOf(SegmentData::place, SegmentData::manner, SegmentData::voiced).random()
        val vowelFeatures = listOf(
            Prop(SegmentData::onGlide),
            Prop(SegmentData::offGlide),
            Prop(SegmentData::nasalized),
            Prop(SegmentData::lengthened),
            Prop(SegmentData::rounded),
            Prop<SegmentData, Boolean>("isHigh") { it.isHigh },
            Prop<SegmentData, Boolean>("isLow") { it.isLow },
            Prop<SegmentData, Boolean>("isBackward") { it.isBackward },
            Prop<SegmentData, Boolean>("isFrontward") { it.isFrontward },
        ).filter { feature -> vowels.groupBy { feature.get(it.data) }.size > 1 }
        if (vowelFeatures.isEmpty()) break
        val vowelFeature = vowelFeatures.random()
        val modelConsonant = consonants.random()
        val modelVowel = vowels.random()
        val vowelSet = vowels.filter { vowelFeature.get(it.data) == vowelFeature.get(modelVowel.data) }

        rules.add(
            SyllableRule(
                "${consonantFeature.name} ${consonantFeature.get(modelConsonant.data)} consonants cannot co-occur with {${vowelSet.joinToString { it.display }}} in ${place.name}"
            ) { syllable ->
                place.get(syllable)
                    .none { consonant -> consonantFeature.get(consonant.data) == consonantFeature.get(modelConsonant.data) } ||
                        syllable.nucleus.none { vowel -> vowel in vowelSet }
            })
    }

    if (vowels.any { it.data.nasalized == true }
        && consonants.any { it.data.manner == Manner.NASAL }
        && random.nextDouble() <= 0.5
    ) random.nextDouble().let {
        val onsetNasal =
            onsetClusters.any { it.size > 0 && (Manner.NASAL in it.last().manner || it.last().offGlide.any { it?.manner == Manner.NASAL }) }
        val codaNasal =
            codaClusters.any { it.size > 0 && Manner.NASAL in it.first().manner }
        when {
            it <= 0.33 && onsetNasal && codaNasal -> rules.add(
                SyllableRule(
                    "Nasalized Vowels only adjacent to Nasals",
                    "restricted_nasal_vowels"
                ) { syllable ->
                    syllable.nucleus.any { it.data.nasalized == true } ==
                            (syllable.onset.lastOrNull()?.data?.manner == Manner.NASAL
                                    || syllable.onset.lastOrNull()?.data?.offGlide?.manner == Manner.NASAL
                                    || syllable.coda.lastOrNull()?.data?.manner == Manner.NASAL)
                })
            it <= 0.66 && codaNasal -> rules.add(
                SyllableRule(
                    "Nasalized Vowels only adjacent to Nasals (Coda)",
                    "restricted_nasal_vowels"
                ) { syllable ->
                    syllable.nucleus.any { it.data.nasalized == true } == (syllable.coda.firstOrNull()?.data?.manner == Manner.NASAL)
                })
            onsetNasal -> rules.add(
                SyllableRule(
                    "Nasalized Vowels only adjacent to Nasals (Onset)",
                    "restricted_nasal_vowels"
                ) { syllable ->
                    syllable.nucleus.any { it.data.nasalized == true } ==
                            (syllable.onset.lastOrNull()?.data?.manner == Manner.NASAL
                                    || syllable.onset.lastOrNull()?.data?.offGlide?.manner == Manner.NASAL)
                })
            else -> {}
        }
    }

    if (maxOnset > 1 || maxCoda > 1) {
        // needs probability analysis
        if (random.nextDouble() <= 0.95) {
            rules.add(SyllableRule("Homorganic Consonant Voicing") { syllable ->
                listOf(syllable.onset, syllable.coda).all { cluster ->
                    cluster.windowed(size = 2) { (a, b) -> a.data.voiced == b.data.voiced || a.data.voiced == null || b.data.voiced == null }
                        .all { it }
                }
            })
        }

        // geminate consonants should be represented phonemically or via cross-syllable rules
        rules.add(SyllableRule("No Geminate Consonants") { syllable ->
            listOf(syllable.onset, syllable.coda).all { cluster ->
                cluster.windowed(size = 2) { (a, b) -> a.display.last() != b.display.first() && a.symbol != b.symbol && a.data != b.data }
                    .all { it }
            }
        })

        // needs probability analysis
        if (random.nextDouble() <= 0.95) {
            rules.add(SyllableRule("All Offglides must be Cluster-final") { syllable ->
                listOf(syllable.onset, syllable.coda)
                    .all { cluster ->
                        cluster.size <= 1 || cluster.take(cluster.size - 1)
                            .all { it.data.consonantGlide == null || it.data.consonantGlide?.manner == Manner.FRICATIVE }
                    }
            })
        }

        // needs probability analysis
        while (random.nextDouble() <= 0.66) {
            val manners = (onsetClusters + codaClusters)
                .filter { it.size >= 2 }.random()
                .windowed(size = 2).random()
                .map { it.manner.random() }
                .sortedByDescending { it.ordinal }

            if (manners[0] == manners[1]) {
                continue
            }

            rules.add(SyllableRule("Homorganic ${manners[0]}-${manners[1]} Consonant Placing") { syllable ->
                listOf(syllable.onset, syllable.coda).all { cluster ->
                    cluster.windowed(size = 2) { (a, b) -> a.data.place == b.data.place || a.data.manner !in manners || b.data.manner !in manners }
                        .all { it }
                }
            })
        }
    }

    return rules.distinctBy { it.name }
}


In [39]:
fun generateSyllable(
    consonants: List<Segment>,
    vowels: List<Segment>,
    onsetClusters: List<List<ConsonantSlot>>,
    codaClusters: List<List<ConsonantSlot>>,
    syllableRules: List<SyllableRule> = listOf()
): Syllable {
    while (true) {
        val onset = onsetClusters.random()
        val nucleus = vowels.random()
        val coda = codaClusters.random()

        val syllable = Syllable(
            onset.map { it.getMatching(consonants).random() },
            listOf(nucleus),
            coda.map { it.getMatching(consonants).random() }
        )

        if (syllableRules.all { it.check(syllable) }) return syllable
    }
}

fun generateAllSyllables(
    consonants: List<Segment>,
    vowels: List<Segment>,
    onsetClusters: List<List<ConsonantSlot>>,
    codaClusters: List<List<ConsonantSlot>>,
    syllableRules: List<SyllableRule>
): List<Syllable> {
    val allOnsetClusters = onsetClusters.flatMap { cluster ->
        cartesianProduct(*cluster.map { slot ->
            slot.getMatching(consonants)
                .toSet()
        }.toTypedArray())
    }.toSet()

    val allCodaClusters = codaClusters.flatMap { cluster ->
        cartesianProduct(*cluster.map { slot ->
            slot.getMatching(consonants)
                .toSet()
        }.toTypedArray())
    }.toSet()

    return cartesianProduct(
        allOnsetClusters,
        vowels.map { listOf(it) }.toSet(),
        allCodaClusters
    ).map { (onset, nucleus, coda) ->
        Syllable(onset, nucleus, coda)
    }.filter { syllable -> syllableRules.all { it.check(syllable) } }
}

In [40]:
import dev.biserman.planet.utils.UtilityExtensions.formatDigits

fun weighSyllables(consonants: List<Segment>, vowels: List<Segment>, syllables: List<Syllable>): Map<Syllable, Double> {
    fun (Segment).glideCount() = listOf(
        this.data.onGlide,
        this.data.offGlide,
        this.data.consonantGlide
    ).count { it != null }

    fun weighSegments(segments: List<Segment>): Map<Segment, Double> {
        val segmentOrdering =
            segments.sortedByDescending { it.prevalence + (random.nextDouble().pow(2) - 0.5) }
        val offset = random.nextDouble(1.5, 3.5)
        val power = random.nextDouble(0.7, 1.3)
        val segmentWeights = segmentOrdering.withIndex().associate { (index, segment) ->
            segment to 1.0 / (index + offset).pow(power)
        }

        return segmentWeights
    }

    val consonantWeights = weighSegments(consonants)
    val vowelWeights = weighSegments(vowels)

    println("top consonants: ${consonantWeights.entries.sortedByDescending { it.value }.take(10).joinToString(", ") { "${it.key.display} (${it.value.formatDigits(2)})" }}")
    println("top vowels: ${vowelWeights.entries.sortedByDescending { it.value }.take(10).joinToString(", ") { "${it.key.display} (${it.value.formatDigits()})" }}")

    fun (Segment).complexity() = 1 + this.glideCount() +
            (if (this.data.lengthened == true) 1 else 0) +
            (if (this.data.nasalized == true) 1 else 0)

    fun (Syllable).complexity() = (
            (if (this.onset.size == 0) 1 else this.onset.sumOf { it.complexity() }) +
                    this.nucleus.sumOf { it.complexity() } +
                    this.coda.sumOf { it.complexity() }) *
            max(1, this.length - 1)

    return syllables.associateWith { syllable ->
        val baseWeight = (syllable.onset.fold(1.0) { acc, it -> acc * consonantWeights.getValue(it) } +
                syllable.nucleus.fold(1.0) { acc, it -> acc * vowelWeights.getValue(it) } +
                syllable.coda.fold(1.0) { acc, it -> acc * consonantWeights.getValue(it) })
        val nullOnsetAdjustment = if (syllable.onset.isEmpty()) consonantWeights.values.random() else 1.0
        baseWeight * nullOnsetAdjustment * random.nextDouble(0.5, 1.5) / syllable.complexity().toDouble()
    }
}

In [41]:
import dev.biserman.planet.language.Depth

class WordTransformation(val name: String, val tag: String = "", val transform: (List<Syllable>) -> List<Syllable>)

fun generateWordTransformations(
    syllables: List<Syllable>,
): List<WordTransformation> {
    val consonants = syllables.flatMap { it.allConsonants }.toSet()
    val vowels = syllables.flatMap { it.nucleus }.toSet()
    val onsetClusters = syllables.map { it.onset }.toSet()
//    val codaClusters = syllables.map { it.coda }.toSet()

    val transformations = mutableListOf<WordTransformation>()

    val glottalStop = SyllableConstructor.segments["ʔ"]!!
    if (random.nextDouble() <= 0.25
        && onsetClusters.any { it.size == 0 }
    ) {
        transformations.add(WordTransformation("Add glottal stops to word-initial vowels") { syllables ->
            val first = syllables.first()
            if (first.onset.isNotEmpty()) syllables else listOf(
                first.copy(onset = listOf(glottalStop).plus(first.onset.drop(1)))
            ).plus(syllables.drop(1))
        })
    }

    if (random.nextDouble() <= 0.25 && vowels.size >= 6) {
        val vowelFeatures =
            listOf(
                Prop(SegmentData::depth),
                Prop(SegmentData::height),
                Prop(SegmentData::nasalized),
                Prop(SegmentData::rounded),
                Prop<SegmentData, Boolean>("isHigh") { it.isHigh },
                Prop<SegmentData, Boolean>("isLow") { it.isLow },
                Prop<SegmentData, Boolean>("isBackward") { it.isBackward },
                Prop<SegmentData, Boolean>("isFrontward") { it.isFrontward },
            ).associateWith { feature -> vowels.groupBy { feature.get(it.data) }.values.filter { it.size > 3 } }
                .filterValues { featureGroups -> featureGroups.size == 2 }

        if (vowelFeatures.isNotEmpty()) {
            val (chosenFeature, groups) = vowelFeatures.entries.random()
            val (firstGroup, secondGroup) = groups

            val foreMap = firstGroup.associateWith { first -> secondGroup.minBy { first.vowelDistanceTo(it) } }
            val aftMap = secondGroup.associateWith { second -> firstGroup.minBy { second.vowelDistanceTo(it) } }

            transformations.add(WordTransformation("Vowel harmony via ${chosenFeature.name}") { syllables ->
                val primaryVowel = (syllables
                    .firstOrNull { syllable -> syllable.nucleus.any { it.data.lengthened == true || it.data.onGlide != null || it.data.offGlide != null } }
                    ?: syllables.first()).nucleus.first()

                when (primaryVowel) {
                    in firstGroup -> syllables.map { syllable ->
                        syllable.copy(nucleus = syllable.nucleus.map {
                            aftMap[it] ?: it
                        })
                    }
                    in secondGroup -> syllables.map { syllable ->
                        syllable.copy(nucleus = syllable.nucleus.map {
                            foreMap[it] ?: it
                        })
                    }
                    else -> syllables
                }
            })
        }
    }

    if (random.nextDouble() <= 0.25) {
        transformations.add(WordTransformation("Interject glottal stop between adjacent vowels", "intervocalic_glottal_stop") { syllables ->
            syllables.drop(1).fold(syllables.take(1)) { acc, syllable ->
                if (acc.last().coda.isEmpty() && syllable.onset.isEmpty()) {
                    acc.plus(syllable.copy(onset = listOf(glottalStop)))
                } else acc.plus(syllable)
            }
        })
    } else {
        transformations.add(WordTransformation("Elide adjacent identical vowels") { syllables ->
            syllables.drop(1).fold(syllables.take(1)) { acc, syllable ->
                val lastVowel = acc.last().nucleus.last()
                val trueLastVowel =
                    if (lastVowel.data.offGlide != null) lastVowel.data.offGlide.vowel(false)!! else lastVowel
                val nextVowel = syllable.nucleus.first()
                val trueNextVowel =
                    if (nextVowel.data.onGlide != null) nextVowel.data.onGlide.vowel(true)!! else nextVowel
                if (acc.last().coda.isEmpty()
                    && syllable.onset.isEmpty()
                    && (trueLastVowel.data.depth == trueNextVowel.data.depth
                            && trueLastVowel.data.height == trueNextVowel.data.height
                            && trueLastVowel.data.rounded == trueNextVowel.data.rounded)
                ) {
                    val canLengthen = vowels.any { it.data.lengthened == true }
                    acc.dropLast(1).plus(
                        acc.last()
                            .copy(
                                nucleus = if (canLengthen) syllable.nucleus.map { it.copy(data = it.data.copy(lengthened = true)) }
                                else syllable.nucleus,
                                coda = syllable.coda
                            )
                    )
                } else acc.plus(syllable)
            }
        })
    }



    return transformations.toList()
}

In [42]:
import dev.biserman.planet.utils.WeightedBag

fun generateWord(
    length: Int,
    syllableBag: WeightedBag<Syllable>,
    wordTransformations: List<WordTransformation>
): List<Syllable> {
    return wordTransformations
        .fold((1..length).map { syllableBag.grab()!! }) { syllables, transformation ->
            transformation.transform(
                syllables
            )
        }
}

In [108]:
class Letter(val lowerCase: String, val titleCase: String, val upperCase: String, val search: String) {
    constructor (lowerCase: String, search: String = lowerCase) : this(
        lowerCase,
        lowerCase.uppercase(),
        lowerCase.uppercase(),
        search
    )

    constructor (lowerCase: String, upperCase: String, search: String = upperCase) : this(
        lowerCase,
        upperCase,
        upperCase,
        search
    )
}

class EnglishOrthographyConfig(
    val enableDiacritics: Boolean = true,
    val enableDiaresis: Boolean = true,
    val enableLigatures: Boolean = true,
    val enableToneMarkers: Boolean = false,
    val enableComplexSyllables: Boolean = false,
    val enableSpecialCharacters: Boolean = false,
)

fun flip() = random.nextDouble() < 0.5

// this is an absolute awful rat's nest. welcome to English vowels!
fun generateEnglishOrthography(
    syllables: List<Syllable>,
    syllableRules: List<SyllableRule>,
    wordTransformations: List<WordTransformation>,
    config: EnglishOrthographyConfig = EnglishOrthographyConfig()
): (List<Syllable>) -> (String) {
    val consonants = syllables.flatMap { it.allConsonants }.toSet()
    val vowels = syllables.flatMap { it.nucleus }.toSet()
    val tones = syllables.mapNotNull { it.tone }.toSet()
    val segments = consonants + vowels

    val vowelGroups = vowels
        .map { it.copy(data = it.data.copy(onGlide = null, offGlide = null)) }
        .groupBy { it.copy(data = it.data.copy(nasalized = false, lengthened = false)) }

    val baseVowels = vowelGroups.values
        .map { it.minBy { (if (it.data.nasalized == true) 2 else 0) + (if (it.data.lengthened == true) 1 else 0) } }

    val baseLetters = mutableMapOf<String, Letter>()

    fun moreVowelsRequired() = baseVowels.any { it.symbol !in baseLetters }

    val yOffGliders =
        segments.filter { it.data.offGlide?.manner == Manner.SEMIVOWEL && it.data.offGlide?.place == Place.PALATAL }
    val yOnGliders =
        segments.filter { it.data.onGlide?.manner == Manner.SEMIVOWEL && it.data.onGlide?.place == Place.PALATAL }
    val vocalicYAvailable =
        yOnGliders.isEmpty() && yOffGliders.isEmpty() && consonants.none { it.symbol == "j" } // $$$ OR j is mapped to j

    val requiresNasalMarking = (syllableRules.none { it.tag == "restricted_nasal_vowels" }
            && vowelGroups.any { it.value.map { it.data.nasalized }.distinct().count() > 1 })
    val nasalDiacriticDesired =
        requiresNasalMarking && syllables.any { it.coda.firstOrNull()?.data?.manner == Manner.NASAL }
    val requiresLengthMarking = vowelGroups.any { it.value.map { it.data.lengthened }.distinct().count() > 1 }

    // if this was the ONLY vowel, what letter would it have? ordered by priority
    val preferredVowels = mapOf(
        "ɪ" to "i",
        "i" to "i",

        "ɛ" to if (config.enableSpecialCharacters && flip()) "ɛ,e" else "e",
        "e̞" to "e",
        "ɘ" to "e",
        "ɜ" to "e",
        "e" to "e",

        "u" to "u",
        "ʉ" to if (vocalicYAvailable) "u,y" else "u",
        "y" to if (vocalicYAvailable) "y,u,i" else "u,i",
        "ɯ" to if (vocalicYAvailable) "u,y" else "u",
        "ɤ" to "u,e",
        "ɤ̞" to "u,e",
        "œ" to "u,e" + if (config.enableLigatures && flip()) ",œ" else "",
        "ø" to "u" + if (config.enableDiacritics && flip()) ",ø" else "",
        "ø̞" to "u" + if (config.enableDiacritics && flip()) ",ø" else "",

        "o" to "o",
        "o̞" to "o",

        "ɵ" to "o",

        "a" to "a",
        "ɐ" to "a",
        "ä" to "a",
        "ɑ" to "a",
        "ɒ" to "a",
        "ʌ" to "a",
        "æ" to "a" + if (config.enableLigatures) ",æ" else "",
        "ɞ" to "a",

        "ʏ" to if (vocalicYAvailable) "y,u,i" else "u,i",
        "ɔ" to "o,a",
        "ʊ" to "u,o",
        "ɨ" to "u,i",
        "ɶ" to "u,a",
        "ə" to "e,u,o,a,i",
    )

    preferredVowels.keys
        .filter { vowel -> baseVowels.any { it.symbol == vowel } }
        .forEach { vowel ->
            val options = preferredVowels[vowel]!!.split(",")
            val chosen = options.firstOrNull { option -> baseLetters.values.none { option in it.lowerCase } }
            if (chosen != null) {
                baseLetters[vowel] = Letter(chosen)
            }
        }

//    val hasPolyphthongs = vowels.any { it.data.offGlide != null || it.data.onGlide != null }
    val adjacentVowelsPossible =
        wordTransformations.none { it.tag == "intervocalic_glottal_stop" } && syllables.any { it.onset.size == 0 }
    val useDiaresis = config.enableDiaresis && adjacentVowelsPossible && flip()
    val canDigraphVowels = !adjacentVowelsPossible || useDiaresis
    val noCodaH = syllables.none { it.coda.firstOrNull()?.symbol == "h" }

    val shouldDigraphVowels = canDigraphVowels && (listOf(
        useDiaresis,
        nasalDiacriticDesired && config.enableDiacritics,
        requiresLengthMarking,
        tones.isNotEmpty()
    ).count { it } <= 2) && flip()
    val lengthDiacriticDesired = config.enableDiacritics && requiresLengthMarking && (shouldDigraphVowels || flip())

    // worst case: diaresis + long + nasal + tone + diff
    // no digraphs: nasal + tone + diff

    val weakI = baseVowels.firstOrNull { it.symbol == "ɪ" } ?: baseVowels.firstOrNull { it.symbol == "ʏ" }
    val strongI = baseVowels.firstOrNull { it.symbol == "i" } ?: baseVowels.firstOrNull { it.symbol == "ɨ" }
    if (weakI != null && strongI != null) {
        if ((shouldDigraphVowels || noCodaH) && (flip() || !config.enableDiacritics)) {
            if ((!noCodaH || flip()) && !requiresLengthMarking) {
                baseLetters[weakI.symbol] = Letter("i")
                baseLetters[strongI.symbol] = Letter("ee", "Ee", "EE")
            } else if (noCodaH) {
                baseLetters[weakI.symbol] = Letter("ih", "Ih", "IH")
                baseLetters[strongI.symbol] = Letter("i")
            }
        } else if (config.enableDiacritics) {
            baseLetters[weakI.symbol] =
                if (config.enableSpecialCharacters && random.nextDouble() <= 0.1) Letter("ɪ", "I")
                else Letter("ı", "I")
            baseLetters[strongI.symbol] = Letter("i", "İ")
        }
    }

    val schwaLike = "ə,ɘ,ɜ,ɐ,ɞ,ɤ,œ".split(",")
    val unallocatedSchwa =
        schwaLike.firstOrNull { schwa -> schwa !in baseLetters && baseVowels.any { it.symbol == schwa } }
    if (unallocatedSchwa != null && noCodaH) {
        baseLetters[unallocatedSchwa] = Letter("uh", "Uh", "UH")
    }

    val aLike = "æ,a,ɐ,ä,ɑ,ɶ".split(",")
    val includedALike = aLike
        .filter { a -> baseVowels.any { it.symbol == a } }
        .filter { baseLetters[it]?.lowerCase != "æ" }
    val combineAllAh = !config.enableLigatures && !config.enableDiacritics
    if (includedALike.size > 1
        && noCodaH
        && (combineAllAh || (flip() && includedALike.size == 2))
    ) {
        baseLetters[includedALike.first()] = Letter("a")
        includedALike.drop(if (combineAllAh && canDigraphVowels) 2 else 1)
            .forEach { baseLetters[it] = Letter("ah", "Ah", "AH") }
    }

    val eLike = "e,e̞,ɛ,ɜ,ä,ɑ,ɶ".split(",")
    val includedELike = eLike
        .filter { e -> baseVowels.any { it.symbol == e } }
    val combineAllEh = !config.enableLigatures && !config.enableDiacritics
    if (includedELike.size > 1
        && noCodaH
        && (combineAllEh || (flip() && includedELike.size == 2))
    ) {
        baseLetters[includedELike.first()] = Letter("e")
        includedELike.drop(if (combineAllEh && canDigraphVowels) 2 else 1)
            .forEach { baseLetters[it] = Letter("eh", "Eh", "EH") }
    }

    if (moreVowelsRequired() && shouldDigraphVowels && (flip() || !config.enableDiacritics)) {
        val preferredVowelDigraphs = mapOf(
            "œ" to "eu",
            "ø" to "eu",
            "ʏ" to "eu",
            "ɘ" to "eu",
            "y" to "eu",

            "ɒ" to "au",
            "ɔ" to "au",
            "ɶ" to "au",
            "ʌ" to "au",

            "æ" to "ae",
            "e" to if (yOffGliders.none { it.symbol == "e" }) "ey" else "ae",

            "u" to "ue",
            "ʉ" to "ue",
            "ɯ" to "ue",
            "ɨ" to "ue",
        ).plus(
            if (!lengthDiacriticDesired) mapOf(
                "ʊ" to "oo",
                "ɤ" to "oo",
                "ɤ̞" to "oo",
                "ø" to "oo",

                "ɑ" to "aa",
                "a" to "aa",
                "ɐ" to "aa",
                "ä" to "aa",
            ) else mapOf()
        )

        preferredVowelDigraphs.keys
            .filter { vowel -> baseVowels.any { it.symbol == vowel } }
            .filter { vowel -> vowel !in baseLetters }
            .forEach { vowel ->
                val options = preferredVowelDigraphs[vowel]!!.split(",")
                val chosen = options.firstOrNull { option -> baseLetters.values.none { option in it.lowerCase } }
                if (chosen != null) {
                    baseLetters[vowel] = Letter(chosen, chosen.first().uppercase() + chosen.drop(1), chosen.uppercase())
                }
            }
    }

    val nasalDiacritic = if (nasalDiacriticDesired && config.enableDiacritics) listOf(
        "◌̃",
        "◌̃",
        "◌̃",
        "◌̃",
        "◌̐"
    ).plus(if (!moreVowelsRequired()) listOf("◌̇", "◌̣") else listOf())
        .random()[1] else null

    val diaresis = if (useDiaresis) "◌̈"[1] else null
    val lengthDiacritic = if (lengthDiacriticDesired) listOf(
        "◌̄",
        "◌̄",
        "◌̄",
        "◌̅",
    ).plus(if (diaresis == null) listOf("◌̈") else listOf())
        .plus(
            if (!moreVowelsRequired()) listOf(
                "◌̀", "◌́", "◌̂", "◌̆", "◌̨", "◌̊"
            ) else listOf()
        )
        .random()[1] else null

    if (moreVowelsRequired() && config.enableDiacritics) {
        val remainingVowels = baseVowels
            .filter { it.symbol !in baseLetters }
            .groupBy { preferredVowels[it.symbol]!!.split(",").random() }
        val diacriticsNeeded = remainingVowels.values.maxOf { it.size }
        val maxDiacritics =
            listOf(useDiaresis, nasalDiacriticDesired, lengthDiacriticDesired, tones.isNotEmpty()).count { it }
        val possibleDiacritics = (if (maxDiacritics >= 3 || random.nextDouble() < 0.33) {
            listOf("◌̜", "◌̝", "◌̞", "◌̣", "◌̥̣", "◌̧", "◌̪", "◌̫", "◌̭", "◌̱", "◌̻")
        } else listOf()).plus(
            if (maxDiacritics <= 2) {
                listOf("◌̀", "◌́", "◌̂", "◌̆", "◌̇", "◌̈", "◌̊", "◌̉")
            } else listOf()
        ).map { it[1] }

        val selectedDiacritics = possibleDiacritics.shuffled(random).take(diacriticsNeeded)
        remainingVowels.forEach { (baseLetter, vowels) ->
            vowels.zip(selectedDiacritics)
                .forEach { (vowel, diacritic) -> baseLetters[vowel.symbol] = Letter(baseLetter + diacritic) }
        }
    }

    if (moreVowelsRequired()) {
        println("Ran out of options, adding ambiguous vowels.")
        baseVowels.filter { it.symbol !in baseLetters }
            .forEach {
                baseLetters[it.symbol] = Letter(preferredVowels[it.symbol]!!)
            }
    }

    val diacriticPlaceFn = random.nextDouble().let {
        when {
            it < 0.25 -> ({ _: Int -> 1 })
            it < 0.65 -> ({ size: Int -> max(1, size / 2 - 1) })
            it < 0.85 -> ({ size: Int -> max(1, size / 2) })
            else -> ({ size: Int -> size })
        }
    }

    val defaultNasalSuffix = if (requiresNasalMarking && !nasalDiacriticDesired) {
        consonants.filter { it.data.manner == Manner.NASAL }.randomOrNull()?.symbol ?: "n"
    } else null

    val hasDoubledVowels = baseLetters.values.any { it.lowerCase.count() == 2 && it.lowerCase.toSet().count() == 1 }

    println("baseLetters: ${baseLetters.mapValues { (_, letter) -> letter.lowerCase }}")

    return ({ word ->
        word.joinToString("") { syllable ->
            syllable.onset.joinToString("") { it.display } +
                    syllable.nucleus.joinToString("") { segment ->
                        val letter = baseLetters[segment.symbol]!!.lowerCase
                        val shouldDouble = segment.data.lengthened == true && requiresLengthMarking && lengthDiacritic == null && !hasDoubledVowels
                        val baseList = listOf(
                            segment.data.onGlide?.display(segment)?.filter { it != "◌̯"[1] },
                            if (shouldDouble) letter.trimEnd('h') else letter,
                            if (shouldDouble) letter else null,
                            if (segment.data.nasalized == true && requiresNasalMarking && !nasalDiacriticDesired) {
                                consonants.firstOrNull {
                                    it.data.manner == Manner.NASAL
                                            && it.data.place == syllable.coda.firstOrNull()?.data?.place
                                }?.symbol ?: defaultNasalSuffix
                            } else null,
                            segment.data.offGlide?.display(segment)?.filter { it != "◌̯"[1] }
                        ).filterNotNull().toMutableList()

                        val applicableDiacritics = listOf(
                            if (segment.data.lengthened == true) lengthDiacritic else null,
                            if (segment.data.nasalized == true) nasalDiacritic else null,
                            if (syllable.tone != null) syllable.tone!!.display else null
                        ).filterNotNull().joinToString("")

                        baseList.add(diacriticPlaceFn(baseList.size), applicableDiacritics)
                        baseList.joinToString("")
                    } + syllable.coda.joinToString("") { it.display }
        }
    })
}

In [109]:
import dev.biserman.planet.utils.UtilityExtensions.formatDigits
import kotlin.random.nextInt

val consonants = generateConsonants()
println("consonants: ${consonants.map { it.display }}")

val vowels = generateVowels()
println("vowels: ${vowels.map { it.display }}")

val onsetMaxConsonants = random.nextDouble().let {
    when {
        it <= 0.2 -> 3
        it <= 0.8 -> 2
        else -> 1
    }
}
val codaMaxConsonants = min(3, onsetMaxConsonants + random.nextInt(-1..1))
println("max onset consonants: $onsetMaxConsonants, coda consonants: $codaMaxConsonants")

val fallOff = random.nextDouble()
println("fall off: $fallOff")

val sonoritySequencingStrictness = random.nextDouble(0.5, 1.0).pow(0.33)
println("sonority sequencing strictness: ${sonoritySequencingStrictness.formatDigits()}")

val (onsetClusters, codaClusters) = generateClusters(
    onsetMaxConsonants,
    codaMaxConsonants,
    fallOff,
    sonoritySequencingStrictness,
    consonants
)
println("onset patterns (${onsetClusters.size}):\n${onsetClusters.joinToString("") { " - $it\n" }}")
println("coda patterns (${codaClusters.size}):\n${codaClusters.joinToString("") { " - $it\n" }}")

val syllableRules =
    generateSyllableRules(consonants, vowels, onsetClusters, codaClusters, onsetMaxConsonants, codaMaxConsonants)
println("syllable rules:\n${syllableRules.joinToString("") { " - ${it.name}\n" }}")


val allSyllables = generateAllSyllables(consonants, vowels, onsetClusters, codaClusters, syllableRules)

val trueOnsets = allSyllables.map { it.onset }.distinct()
val trueCodas = allSyllables.map { it.coda }.distinct()

println("onsets (${trueOnsets.size}): ${trueOnsets.map { it.joinToString("") { it.display } }}")
println("codas (${trueCodas.size}): ${trueCodas.map { it.joinToString("") { it.display } }}")

println(
    "sample syllables: ${
        (1..20).map { allSyllables.random().toString() }.joinToString()
    }"
)

val syllableWeights = weighSyllables(consonants, vowels, allSyllables)
println(
    "top 20 syllables:\n${
        syllableWeights.toList()
            .sortedByDescending { it.second }
            .take(20)
            .joinToString("") { " - ${it.first}: ${it.second.formatDigits()}\n" }
    }"
)

val wordTransformations = generateWordTransformations(allSyllables)
val syllableBag = syllableWeights.keys.toWeightedBag(random) { syllableWeights[it]!! }
val averageWordLength = 20.0 / log(allSyllables.size.toDouble(), 2.0)

if (wordTransformations.isEmpty()) println("No word transformations.\n")
else {
    println("word transformations:\n${wordTransformations.joinToString("") { " - ${it.name}\n" }}")
    println(
        "example transformations:\n${
            (1..10).map {
                val word = (1..ceil(averageWordLength).toInt()).map { syllableBag.grab()!! }
                word.joinToString("") to wordTransformations
                    .fold(word) { word, transformation -> transformation.transform(word) }
                    .joinToString("")
            }.joinToString("") {
                " - ${it.first} → ${it.second}\n"
            }
        }"
    )
}

val orthography = generateEnglishOrthography(
    allSyllables, syllableRules, wordTransformations, EnglishOrthographyConfig(
        enableDiacritics = true,
        enableDiaresis = true,
        enableLigatures = true,
        enableToneMarkers = true,
        enableComplexSyllables = true,
        enableSpecialCharacters = true
    )
)

val sentences = (1..5).map {
    (1..random.nextInt(
        4,
        12
    )).map {
        generateWord(
            (averageWordLength * (random.nextDouble() * random.nextDouble()).scaleAndCoerceIn(
                0.0..1.0,
                0.5..1.5
            )).roundToInt(), syllableBag, wordTransformations
        )
    }
}.associateWith { it.map { orthography.invoke(it) } }

println(
    "example sentences:\n${
        sentences.entries.joinToString("") { (ipa, ortho) ->
            ipa.joinToString(" ") { it.joinToString("") } + " → " + ortho.joinToString(" ") + "\n"
        }
    }"
)

println(
    "total possible syllables (${allSyllables.size}): ${
        allSyllables.map { it.toString() }
            .sorted()
            .joinToString()
    }"
)


consonants: [m, n, ŋ, p, p?, t, t?, k, kʷ, k?, d, d?, ɡ, ɡ?, f?, s, s?, j, w, l]
vowels: [i, u, e, o, ɛ, a, iː, uː, eː, oː, aː]
max onset consonants: 3, coda consonants: 2
fall off: 0.8054296103724915
sonority sequencing strictness: 0.91
onset patterns (5):
 - [ConsonantSlot(manner=[NASAL, PLOSIVE, FRICATIVE, SEMIVOWEL, LIQUID], place=[LABIAL, ALVEOLAR, VELAR], voiced=[null, false, true], consonantGlide=[null, Glide(place=ALVEOLAR, manner=SEMIVOWEL, isOnGlide=false), Glide(place=LABIAL, manner=SEMIVOWEL, isOnGlide=false)], offGlide=[null])]
 - []
 - [ConsonantSlot(manner=[PLOSIVE], place=[LABIAL, VELAR], voiced=[false], consonantGlide=[null], offGlide=[null]), ConsonantSlot(manner=[NASAL], place=[LABIAL, ALVEOLAR], voiced=[null, false], consonantGlide=[null], offGlide=[null])]
 - [ConsonantSlot(manner=[FRICATIVE], place=[LABIAL, ALVEOLAR, VELAR], voiced=[false, null, true], consonantGlide=[Glide(place=ALVEOLAR, manner=SEMIVOWEL, isOnGlide=false), null], offGlide=[null]), ConsonantSlot(